# NFL Money Line Analyzer #

This Notebook uses the season stats of NFL teams to predict the outcome of match ups. The data is then run through a Decision Tree, Random Forest, and Naive Bayes classifiers.

The data is sourced from footballdb.com

CSV files besides matchup2.csv must have team names in alphabetical order

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics  
from sklearn import tree
import numpy as np
from matplotlib import pyplot as plt

#### Data sourced from Footballdb.com ####
pass_df = pd.read_csv('PassRippedData.csv')
rush_df = pd.read_csv('RushRippedData.csv') 
passD_df = pd.read_csv('Pass_DRippedData.csv') 
rushD_df = pd.read_csv('Rush_DRippedData.csv')
matchup_df = pd.read_csv('matchup2.csv')
teamlist_df = pd.read_csv('teamlist.csv')

teamlist_df = teamlist_df.drop('Unnamed: 5', axis=1)

# Data Cleaning Functions #

The function MoreIsBetter is best used for simplifying team wins, postive offensive production (yards gained, first downs, touchdowns), and defensive stats (3rd down stops, turnovers).

The function LessIsBetter is best used for simplifying negative offensive production (interceptions, fumbles, 3 and outs) and positive defensive production (passing/rushing yards allowed, touchdowns allowed)

In [13]:
def MoreIsBetter(stat_df, col_name):
    """
    stat_df_c: pass a column of a data frame
    col_name: column name of stat to be simplified
    
    return: column with stats replaced with 1 if stat is above league average.
            0 if stat is below league average. 
    """

    stat_pg = stat_df[col_name]

    avg_stat = 0
    for x in stat_pg:
        avg_stat += int(x)

    avg_stat = int(avg_stat / 32)

    for x in stat_pg:
        if x > avg_stat:
            stat_pg.replace(x, 1, inplace=True)

    for x in stat_pg:
        if x > 2:
            stat_pg.replace(x, 0, inplace=True)

def LessIsBetter(stat_df, col_name):
    """
    stat_df_c: pass a column of a data frame
    col_name: column name of stat to be simplified
    
    return: column with stats replaced with 1 if stat is above league average.
            0 if stat is below league average. 
    """

    stat_pg = stat_df[col_name]

    avg_stat = 0
    for x in stat_pg:
        avg_stat += int(x)

    avg_stat = int(avg_stat / 32)

    for x in stat_pg:
        if x < avg_stat:
            stat_pg.replace(x, 1, inplace=True)

    for x in stat_pg:
        if x > 2:
            stat_pg.replace(x, 0, inplace=True)

# Data Frame Combination #

Takes the binary data created from MoreIsBetter and LessIsBetter to create a single data frame.

Current Version: Not designed to use additional stats outside of the original four 

Next Version: Will use the panda add column feature to create a dataframe 

In [14]:
def combination():
    """
    Current version no parameters, no returns
    
    Next version:
    teamlist_df
    column to be added 
    """
    teamlist_df['Above Average Rush O'] = rush_df['Yds/G']
    teamlist_df['Above Average Pass O'] = pass_df['Yds/G']
    teamlist_df['Above Average Rush D'] = rushD_df['Yds/G']
    teamlist_df['Above Average Pass D'] = passD_df['Yds/G']

    teamlist_df = teamlist_df.set_index('Team')

    matchup_rushO = []
    matchup_passO = []
    matchup_rushD = []
    matchup_passD = []

    for x in matchup_df['Home Team']:
        matchup_rushO.append(teamlist_df.loc[x,'Above Average Rush O'])
        matchup_passO.append(teamlist_df.loc[x,'Above Average Pass O'])
        matchup_rushD.append(teamlist_df.loc[x,'Above Average Rush D'])
        matchup_passD.append(teamlist_df.loc[x,'Above Average Pass D'])
    
    matchup_df['HT Above Average Rush O'] = matchup_rushO
    matchup_df['HT Above Average Pass O'] = matchup_passO
    matchup_df['HT Above Average Rush D'] = matchup_rushD
    matchup_df['HT Above Average Pass D'] = matchup_passD

    matchup_rushO = []
    matchup_passO = []
    matchup_rushD = []
    matchup_passD = []

    for x in matchup_df['Away Team']:
        matchup_rushO.append(teamlist_df.loc[x,'Above Average Rush O'])
        matchup_passO.append(teamlist_df.loc[x,'Above Average Pass O'])
        matchup_rushD.append(teamlist_df.loc[x,'Above Average Rush D'])
        matchup_passD.append(teamlist_df.loc[x,'Above Average Pass D'])
    
    matchup_df['AT Above Average Rush O'] = matchup_rushO
    matchup_df['AT Above Average Pass O'] = matchup_passO
    matchup_df['AT Above Average Rush D'] = matchup_rushD
    matchup_df['AT Above Average Pass D'] = matchup_passD

# Main #

In [15]:
MoreIsBetter(pass_df, 'Yds/G')
MoreIsBetter(rush_df, 'Yds/G')
LessIsBetter(passD_df, 'Yds/G')
LessIsBetter(rushD_df, 'Yds/G')

combination()
matchup_df

UnboundLocalError: local variable 'teamlist_df' referenced before assignment